# PBL01_需要予測・在庫最適化　サンプルコード

当サンプルコードは、「需要予測・在庫最適化のテーマのAI課題」において配布されたデータを分析し、投稿ファイルを作成するまでの大まかな流れを、ソースコードや実行結果とともに示したものとなっています。詳細な分析を行う足がかりとしてご利用ください。<br>

なお、当サンプルコードでは、プログラミング言語Pythonを使用します。<br>
これまでにPythonを使ったことが無いといった方々に関しては、事前に下記の講座を受講していただくことをおすすめします。<br>

- [SIGNATE Cloud講座「Python入門」](https://biz.quest.signate.jp/quests/10003)
<br>(※ 受講生の皆様は、マナビDXQuestの期間中、SIGNATE Cloudを無料で受講することができます。)

当サンプルコードは、以下の流れで構成されています。

1. [初期設定](#0)
1. [ライブラリのインポート](#1)
1. [データの読み込み](#2)
1. [データの概要把握](#3)
1. [前処理](#4)
1. [モデリング](#5)
1. [提出用ファイルの作成](#6)
1. [精度改善に向けたヒント](#7)

必要なライブラリは以下です。

- os
- pandas（1.5.3）
- numpy（1.23.5）
- matplotlib
- seaborn
- scikit-learn（1.2.2）

使用するデータは以下です。

- sales_history.csv
- item_categories.csv
- category_names.csv
- test.csv
- sample_submission.csv

サイトからダウンロードして当ノートブックと同じディレクトリに配置してください。なお、以下のようなフォルダ構成を前提としています。

```
PBL01（Colabの場合は"DXQuest"になっている想定）
└─ PBL01_sample_code.ipynb（本notebook）
└─ sales_history.csv
└─ item_categories.csv
└─ category_names.csv
└─ test.csv
└─ sample_submission.csv
```

<a name="0"></a>
## 0. 初期設定
- こちらはColabを利用される際の初期設定となります。Colabを利用されない方はこちらの実行は不要です。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- ファイルパスの設定
    - 利用される環境に応じて片方をコメントアウト、片方を実行する形で修正ください。
    - 下記ではGoogle Colaboratoryで実行する想定としています。

In [2]:
import os

## Google Colabを利用する場合
path = '/content/drive/MyDrive/DXQuest/'

## ローカルの場合
# path = "./"

In [3]:
# 想定している場所にファイルがあるかどうかの確認
# 出力想定は下記の通り（順不同）
# ['PBL05_sample_code.ipynb','sample_submission.csv','category_names.csv','test.csv','sales_history.csv','item_categories.csv']
os.listdir(path)

['sample_submission.csv',
 'category_names.csv',
 'test.csv',
 'sales_history.csv',
 'item_categories.csv',
 'PBL01_sample_code.ipynb']

<a name="1"></a>
# 1. ライブラリのインポート

はじめに「ライブラリ」をインポートします。<br>
「ライブラリ」とは、特定の用途において便利な機能を持つ道具箱のようなものです。<br>
Pythonには、『表形式のデータを扱う』ためのライブラリ、『グラフを綺麗に描く』ためのライブラリ、『機械学習モデルを構築する』ためのライブラリ等、データ分析を行う上でとても便利なライブラリがたくさん存在しており、それがプログラミング言語としての人気の要因になっています。

※ 厳密には、「ライブラリ」の他にも「モジュール」という概念があり、両者はそれぞれ少しだけ意味が異なります。ただし、最初の内は特にこれらの違いを意識する必要はないため、ここでは簡単のため「ライブラリ」に統一します。

In [4]:
# Pandas: 表形式のデータを扱うためのライブラリ
import pandas as pd

# NumPy: 多次元配列の演算を高速に行うためのライブラリ
import numpy as np

# matplotlib, seaborn: グラフを描くためのライブラリ
from matplotlib import pyplot as plt
import seaborn as sns

# scikit-learn: 機械学習モデルを構築するためのライブラリ
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

これ以降は、上記のライブラリの便利な機能を活用しながら分析を進めていきます。<br>
これらのライブラリの基本的な使い方に関しては、下記の講座で詳しく説明されています。<br>
必要に応じて、学習にお役立てください。

- Pandasの使い方に関する講座
    - [SIGNATE Cloud講座「Pandas入門道場」](https://biz.quest.signate.jp/quests/10007)
- NumPyの使い方に関する講座
    - [SIGNATE Cloud講座「NumPy道場」](https://biz.quest.signate.jp/quests/10033)
- matplotlib, seabornの使い方に関する講座
    - [SIGNATE Cloud講座「データ可視化道場」](https://biz.quest.signate.jp/quests/10014)
- scikit-learnの使い方に関する講座
    - [SIGNATE Cloud講座「scikit-learn道場」](https://biz.quest.signate.jp/quests/10076)

<a name="2"></a>
# 1. データの読み込み

本PBLで使用するデータは、全てCSV形式のファイルです。<br>
まずはこれらのCSVファイルを読み込み、プログラム上で扱えるようにましょう。<br>

CSVファイルを読み込むには、pandasライブラリの`read_csv()`関数を使用します。<br>
(※ 下記のデータを読み込む際は、全ての配布ファイルが当サンプルコードと同じディレクトリ内に存在する必要がありますので、あらかじめご確認ください。)

In [5]:
sales = pd.read_csv(path + 'sales_history.csv')
cats = pd.read_csv(path + 'item_categories.csv')
cat_names = pd.read_csv(path + 'category_names.csv')
test = pd.read_csv(path + 'test.csv', index_col=0)
sub = pd.read_csv(path + 'sample_submission.csv', header=None)

<a name="3"></a>
# 2. データの概要把握

次に、読み込んだ各ファイルの中身にどのようなデータが含まれているのかを確認していきましょう。

## 2-1. データの表示

まずは、読み込んだ表データをそのまま表示します。<br>
例えば上の読み込み処理では、「sales_history.csv」を「sales」という変数に格納したので、そのまま「sales」と打ち込みプログラムを実行することで、「sales_history.csv」の中身を画面上に表示することができます。

In [6]:
# sales(sales_history.csv)の中身を表示
sales

,日付,店舗ID,商品ID,商品価格,売上個数
0,2018-01-01,9,1000001,420,1.0
1,2018-01-01,6,1000001,420,1.0
2,2018-01-01,10,1000001,420,1.0
3,2018-01-01,0,1000017,250,1.0
4,2018-01-01,1,1000017,250,1.0
...,...,...,...,...,...
1119565,2019-10-31,8,3400006,2700,1.0
1119566,2019-10-31,7,3400009,1020,1.0
1119567,2019-10-31,7,3400012,510,1.0
1119568,2019-10-31,4,3500001,340,1.0


salesデータには、各「店舗」の「商品別」の売上が「日単位」に記録されているようです。
<br>他のデータ(「cats」「cat_names」「test」「sub」)に関しても、同じように中身を確認してみましょう。

In [7]:
# cats(item_categories.csv)の中身を表示
cats

,商品ID,商品カテゴリID
0,1000001,100
1,1000002,100
2,1000003,100
3,1000004,100
4,1000005,100
...,...,...
9421,3500072,350
9422,3500073,350
9423,3500074,350
9424,3500075,350


In [8]:
# cat_names(category_names.csv)の中身を表示
cat_names

,商品カテゴリID,商品カテゴリ名
0,100,映画 - DVD
1,110,PCゲーム - 通常版
2,120,音楽 - その他
3,130,ゲーム - ゲーム機C
4,140,映画 - ブルーレイ
5,150,ゲーム - ゲーム機B
6,160,ギフト - 小型ゲーム
7,170,ギフト - カード、パズル等
8,180,ギフト - フィギュア
9,190,PCソフト - PCソフト


In [9]:
# test(test.csv)の中身を表示
test

,商品ID,店舗ID
index,,
0,1000001,0
1,1000001,1
2,1000001,2
3,1000001,3
4,1000001,4
...,...,...
3055,3500001,13
3056,3500001,14
3057,3500001,15


In [10]:
# sub(sample_submit.csv)の中身を表示
sub

,0,1
0,0,100
1,1,100
2,2,100
3,3,100
4,4,100
...,...,...
3055,3055,100
3056,3056,100
3057,3057,100
3058,3058,100


## 2-2. データの要約情報の表示

上では、表データの中身をそのまま画面上に表示することができました。<br>
ただしデータ量が多いもの関しては、一部のデータが省略して表示されていたりと、まだまだデータの全体像をつかめたとは言い切れません。<br>
そこで、各データの概観をより正確に把握するために、他のやり方で情報を抽出することを考えます。<br>
具体的には、

- データのサイズ
    - 何行何列のデータなのか？
- 欠損値の数
    - 各列には、有効な値 (空欄ではない値) がどの程度含まれているのか？
- データ型
    - 各列の値は「文字データ」なのか「数値データ」なのか「日付データ」なのか？

といった表データが持つ基本的な属性情報を把握したいです。<br>
これらの情報は、pandasライブラリの`.info()`関数を使用することで、画面上に表示することができます。

In [11]:
# sales(sales_history.csv)に関する要約情報を表示
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119570 entries, 0 to 1119569
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   日付      1119570 non-null  object 
 1   店舗ID    1119570 non-null  int64  
 2   商品ID    1119570 non-null  int64  
 3   商品価格    1119570 non-null  int64  
 4   売上個数    1119570 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 42.7+ MB


例えば、上のsalesデータであれば、

- **1119570行 × 5列** の表データであること
- **欠損値 (空欄) に該当するデータは存在せず**、全て有効な値によって構成されていること
- 1列の**浮動小数点(float)型**データ、3列の**整数(int)型**データ、1つの**文字列(object)型データ**によって構成されていること

がわかりました。<br>


同様に、その他のデータに関しても概要情報を表示しましょう。

In [12]:
# cats(item_categories.csv)に関する要約情報を表示
cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   商品ID      9426 non-null   int64
 1   商品カテゴリID  9426 non-null   int64
dtypes: int64(2)
memory usage: 147.4 KB


In [13]:
# cat_names(cat_names)に関する要約情報を表示
cat_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   商品カテゴリID  26 non-null     int64 
 1   商品カテゴリ名   26 non-null     object
dtypes: int64(1), object(1)
memory usage: 544.0+ bytes


In [14]:
# test(test.csv)に関する情報を表示
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3060 entries, 0 to 3059
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   商品ID    3060 non-null   int64
 1   店舗ID    3060 non-null   int64
dtypes: int64(2)
memory usage: 71.7 KB


In [15]:
# sub(sample_submission.csv)に関する要約情報を表示
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       3060 non-null   int64
 1   1       3060 non-null   int64
dtypes: int64(2)
memory usage: 47.9 KB


<a name="4"></a>
# 3. 前処理

ここからは予測モデルを構築するための準備として、データの加工を行います。(こういった処理を一般に**前処理**といいます。)<br>
具体的には、以下の処理を順番に実行していきます。

- **1. 売上の計上単位を「月単位」に統一する**
    - 当課題の目的は「月単位」の売上個数を予測することですが、salesデータにおける売上個数は、冒頭で確認したとおり「日単位」で記録されています。そこで、両者の単位を揃えるために、salesデータを「月単位」で集計するという処理を事前に行います。

- **2. データを統合する**
    - 当課題の配布データは、情報が複数のファイルに分かれています。モデルによる学習・推論を行うには、必要な情報を一つの表に集約しておく必要があるので、予測に役立ちそうなデータに関しては、データの統合処理を行いましょう。
- **3. データを数値化する**
    - モデルに入力するデータは数値型のデータである必要があります。ここで、データの型を全て数値型に変更しましょう。
- **4. 学習用データとテスト用データの構造を統一する**
    - 手順1, 2, 3では、学習用データのみを対象に前処理を行ってきました。一方で、モデルによる学習・推論を正しく行うには、テスト用データの構造を学習用データの構造と統一する必要があリます。そこで、テスト用データに関しても必要なデータ加工を施しましょう。

## 3-1. 売上の計上単位の統一

当課題では「月単位」の売上を予測することが目的のため、過去の売上データに関しても「月単位」で情報を集約する必要があります。<br>
今回は、その前段階として「日付」カラムから年・月の情報を抽出することで、「月単位」の売上個数の集計を行うための準備を行います。<br>
なお、「日付」カラムには '2018-01-01' のように 『年月日が ハイフン'-'で区切られている』という規則性があるため、この規則性を利用して、年・月情報を抽出した新しいカラムを生成しましょう。

ここでは、pandasライブラリの`apply()`関数と、文字列型データに対して適用できる`split()`という2つの関数を使用して、上記の処理を実現します。

※ 参考教材へのリンク... [apply()関数の使い方](https://biz.quest.signate.jp/quests/10010/contents/8/5/1), [split()関数の使い方](https://biz.quest.signate.jp/quests/10003/contents/1/4/2)

In [16]:
# 「日付」カラムの文字列から、「年」「月」の情報を抽出する
sales['年'] = sales['日付'].apply(lambda x: x.split('-')[0])
sales['月'] = sales['日付'].apply(lambda x: x.split('-')[1])

sales.head()

,日付,店舗ID,商品ID,商品価格,売上個数,年,月
0,2018-01-01,9,1000001,420,1.0,2018,01
1,2018-01-01,6,1000001,420,1.0,2018,01
2,2018-01-01,10,1000001,420,1.0,2018,01
3,2018-01-01,0,1000017,250,1.0,2018,01
4,2018-01-01,1,1000017,250,1.0,2018,01


<br>

続いて、上で作成した2つのカラム「年」「月」を利用して、「月単位」の売上個数を集計します。

使用する関数は、pandasライブラリの`groupby()`です。<br>
(予測の対象は、「商品×店舗」ごとの売上個数なので、商品IDと店舗IDの値も`groupby()`の引数に含めます。

※ 参考教材へのリンク... [groupby()の使い方](https://biz.quest.signate.jp/quests/10007/contents/5/3/1)

In [17]:
sales_month = sales.groupby(['商品ID', '店舗ID', '年', '月']).agg({'売上個数': 'sum'}).reset_index()
sales_month.head()

,商品ID,店舗ID,年,月,売上個数
0,1000001,0,2018,01,6.0
1,1000001,0,2018,02,3.0
2,1000001,0,2018,03,1.0
3,1000001,0,2018,04,2.0
4,1000001,0,2018,06,4.0


## 3-2. データを統合する

続いて、複数ファイルにバラバラになっている情報を一つの表に集約する処理を行います。<br>
配布データには、「商品カテゴリ」に関する情報が含まれていますが、その所在はitem_categories.csv(cats)とうファイルにあり、売上データであるsales_histries.csv(sales)とは別の表です。<br>
売上データ(sales)と商品カテゴリデータ(cats)には、共通データとして「商品ID」カラムを持っているため、このカラムが同じIDに該当する行どうしを紐付けることで2つの表を統合することができます。

こちらの統合処理には、pandasライブラリの`merge()`関数を使用します。<br>

※ 参考教材へのリンク... [merge()関数の使い方](https://biz.quest.signate.jp/quests/10007/contents/4/4/1)

In [18]:
# 売上(sale)データと商品カテゴリID(cats)データの統合
sales_month = pd.merge(sales_month, cats, on='商品ID', how='left')

sales_month.head()

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID
0,1000001,0,2018,01,6.0,100
1,1000001,0,2018,02,3.0,100
2,1000001,0,2018,03,1.0,100
3,1000001,0,2018,04,2.0,100
4,1000001,0,2018,06,4.0,100


「商品ID」「商品カテゴリID」に基づき表データを1つに統合することで、売上レコードに対してより豊富な情報を付与することができました。

## 3-3. データを数値化する

予測モデルにデータを入力する際は、データが数値型である必要があります。<br>
「日付」カラムから生成した「年」「月」に関しては、現在文字列型になっているので、これらを整数型intに修正します。<br>

データ型の変更処理には、pandasライブラリの`astype()`関数を使用します。

In [19]:
sales_month['年'] = sales_month['年'].astype('int')
sales_month['月'] = sales_month['月'].astype('int')

sales_month.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495134 entries, 0 to 495133
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   商品ID      495134 non-null  int64  
 1   店舗ID      495134 non-null  int64  
 2   年         495134 non-null  int64  
 3   月         495134 non-null  int64  
 4   売上個数      495134 non-null  float64
 5   商品カテゴリID  495134 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 26.4 MB


無事、全てのデータを数値型(整数intもしくは浮動小数点数float)に統一することができました。<br>

## 3-4. 学習用データとテスト用データの構造を統一する

前処理の締めくくりとして、テスト用データの加工を行うことで学習用データとデータの構造を揃えます。<br>
まずは、改めて学習用データ(sales_month)とテスト用データ(test)を表示することで、両者の違いを確認しましょう。

In [20]:
# 学習用データの表示
sales_month

,商品ID,店舗ID,年,月,売上個数,商品カテゴリID
0,1000001,0,2018,1,6.0,100
1,1000001,0,2018,2,3.0,100
2,1000001,0,2018,3,1.0,100
3,1000001,0,2018,4,2.0,100
4,1000001,0,2018,6,4.0,100
...,...,...,...,...,...,...
495129,3500070,16,2019,10,1.0,350
495130,3500071,13,2019,10,1.0,350
495131,3500072,2,2019,10,1.0,350
495132,3500072,16,2019,10,1.0,350


In [21]:
# テスト用データの表示
test

,商品ID,店舗ID
index,,
0,1000001,0
1,1000001,1
2,1000001,2
3,1000001,3
4,1000001,4
...,...,...
3055,3500001,13
3056,3500001,14
3057,3500001,15


学習用のデータと比べると、「商品ID」と「店舗ID」が共通して存在する一方で、足りないカラムが「年」「月」「売上個数」「商品カテゴリID」の4つあることがわかります。
そこで、これらのテスト用データに足りないカラムに関する対応の方針を考えましょう。

- 「年」... 今回売上を予測したい期間は、『2019年の12月』なので、一律で **2019** となるカラムを作成することで学習用データと条件を揃えることができます。
- 「月」... 「年」カラムと同様の理由で、 一律で **12** とします。
- 「商品カテゴリID」... 学習用データと同様に、共通カラム「商品ID」に基づき **商品カテゴリ(cats)データと統合** しましょう。
- 「売上個数」... これは、『予測をしたい値』であり、モデルに入力する値には含まれないため、**新たにカラムを作成する必要はありません。**

In [22]:
# 「年」カラムの追加
test['年'] = 2019
# 「月」カラムの追加
test['月'] = 12
# 「商品カテゴリID」カラムの追加
test = pd.merge(test, cats, on='商品ID', how='left')

test

,商品ID,店舗ID,年,月,商品カテゴリID
0,1000001,0,2019,12,100
1,1000001,1,2019,12,100
2,1000001,2,2019,12,100
3,1000001,3,2019,12,100
4,1000001,4,2019,12,100
...,...,...,...,...,...
3055,3500001,13,2019,12,350
3056,3500001,14,2019,12,350
3057,3500001,15,2019,12,350
3058,3500001,16,2019,12,350


テスト用データに対して、「年」「月」「商品カテゴリ」のデータを正しく追加することができました。

<a name="5"></a>
# 4. モデリング

次は、いよいよモデリングの段階に移ります。<br>
はじめに、モデルに与えるデータを説明変数(予測の手がかりとなる値)と目的変数(予測の対象となる値)に分割します。

In [23]:
# 説明変数に該当するカラムの一覧
feature_columns = ['商品ID', '店舗ID', '商品カテゴリID', '年', '月']

# 学習用データの整理
X_train = sales_month[feature_columns] # 学習用データの説明変数
y_train = sales_month['売上個数'] # 学習用データの目的変数

# テスト用データの整理
X_test = test[feature_columns] # テスト用データの説明変数

In [24]:
X_train

,商品ID,店舗ID,商品カテゴリID,年,月
0,1000001,0,100,2018,1
1,1000001,0,100,2018,2
2,1000001,0,100,2018,3
3,1000001,0,100,2018,4
4,1000001,0,100,2018,6
...,...,...,...,...,...
495129,3500070,16,350,2019,10
495130,3500071,13,350,2019,10
495131,3500072,2,350,2019,10
495132,3500072,16,350,2019,10


In [25]:
X_test

,商品ID,店舗ID,商品カテゴリID,年,月
0,1000001,0,100,2019,12
1,1000001,1,100,2019,12
2,1000001,2,100,2019,12
3,1000001,3,100,2019,12
4,1000001,4,100,2019,12
...,...,...,...,...,...
3055,3500001,13,350,2019,12
3056,3500001,14,350,2019,12
3057,3500001,15,350,2019,12
3058,3500001,16,350,2019,12


In [26]:
y_train

0         6.0
1         3.0
2         1.0
3         2.0
4         4.0
         ... 
495129    1.0
495130    1.0
495131    1.0
495132    1.0
495133    1.0
Name: 売上個数, Length: 495134, dtype: float64

続いて、モデルの学習フェーズです。<br>
ここでは、「決定木」というシンプルな予測モデルを使用します。<br>
上で作成した学習用データ(X_train, y_train)をモデルに与えることで学習を実行しましょう。

決定木モデルの生成には、scikit-learnライブラリの`DecisionTreeRegressor()`、モデルの学習には`fit()`を使用します。<br>

※ 参考教材へのリンク... [scikit-learn道場](https://biz.quest.signate.jp/quests/10076)

In [27]:
# モデルの型の生成
model = DecisionTreeRegressor()

# モデルの学習
model.fit(X_train, y_train)

DecisionTreeRegressor()

これでモデルの学習が完了しました。<br>
次は推論(予測)フェーズです。<br>
次に、上で学習したモデルに対してテスト用データ(X_test)を与えることで売上個数の予測値を出力しましょう。<br>

モデルの予測には、scikit-learnライブラリの`predict()`関数を使用します。

In [28]:
# 学習済みのモデルを使用してテストデータに対する予測値を算出する
y_pred = model.predict(X_test)
y_pred

array([2., 2., 1., ..., 1., 1., 1.])

テスト用データの各行に対して、モデルによる予測結果を得ることができました。

<a name="6"></a>
# 5. 提出用ファイルの作成


最後に、予測値が埋め込まれた提出用ファイルを作成します。<br>
提出用ファイルのフォーマットは、変数subとして読み込んだsample_submission.csvファイルと揃える必要があるので、ここでsubの中身を改めて見てみましょう。

In [29]:
# sample_submissionの中身を確認
sub

,0,1
0,0,100
1,1,100
2,2,100
3,3,100
4,4,100
...,...,...
3055,3055,100
3056,3056,100
3057,3057,100
3058,3058,100


予測値は、右端の列に入力します。

In [30]:
# sample_submissionの右端のカラムに予測値を代入する。
sub[sub.columns[-1]] = y_pred

sub

,0,1
0,0,2.0
1,1,2.0
2,2,1.0
3,3,2.0
4,4,12.0
...,...,...
3055,3055,3.0
3056,3056,1.0
3057,3057,1.0
3058,3058,1.0


最後に、subをCSVファイルとして保存することで、提出ファイルを生成します。<br>
CSVファイルの保存には、pandasの`to_csv()`関数を使用します。

※参考教材へのリンク... [to_csv()関数の使い方](https://biz.quest.signate.jp/quests/10010/contents/2/4/1)

In [31]:
# 提出ファイルの生成
sub.to_csv('my_submission.csv', index=False, header=False)

予測結果が格納されたCSVファイルを「my_submission.csv」という名前で保存することができました。<br>
まだこれまでに一度も投稿経験がないという方は、こちらのファイルを一回目の投稿としてサイトに提出してみましょう。

<a name="7"></a>
# 6. 精度改善に向けたヒント

当サンプルコードでは、予測モデルを構築し提出ファイルを作成するまでの流れを説明してきました。<br>ただし、作成したモデルはあくまで最低限の処理のみを施したものであり、予測の精度はまだまだお世辞にも高いとは言えません。<br>
そこで、以下により高い精度を達成するための取っ掛かりとなるヒントを記載します。<br>これらの情報を参考にしてモデルの精度改善に挑戦してみましょう。

## 精度向上のための工夫例

- **特徴量を追加する**
    - モデルが予測の手がかりとする情報を増やすことで、より高い予測精度を達成することが期待できます。
    - 「ある月の売上に関連があるのはどんなデータなのか」を考え、予測の手がかりになりそうなデータを説明変数Xとして追加してみましょう。
    - 例えば、年末のクリスマスや年始の初売りなど、「セールが盛んなシーズンは他の時期と比べて売上が多くなるのではないか」という仮説を立てたとします。この仮説が正しい場合は、「1年前の同月、同商品の売上個数」という情報が、予測精度の向上に寄与するかもしれません。
- **予測モデルを変更する**
    - 上のサンプルコードでは「決定木」というモデルを採用して予測を行いましたが、当然他のモデルを使用することで予測精度は変化します。
    - どのモデルが必ずしも「良い」「悪い」ということはなく、データの性質とモデルの特性の組み合わせによって最適解は異なります。
    - 「決定木」以外のモデルとしては、下記のようなものが挙げられます。それぞれのモデルの特性や使い方を調べて精度を検証してみましょう
        - 「重回帰モデル」
        - 「ランダムフォレスト」
        - 「ニューラルネットワーク」
        
        
## 参考教材

SIGNATE Cloudでは、予測課題に関する様々なケーススタディ教材を提供しています。<br>
以下に、当課題に取り組む上で参考になるであろう教材をピックアップしました。<br>
どれもマナビDXQuestの受講期間中は無料で受講できる教材となっておりますので、余裕のある方はお取り組みください。

- [食品ロスの削減](https://biz.quest.signate.jp/quests/10002)
    - お弁当の「廃棄ロス」や「売上機械の損失」を避けるための手段として、未来のお弁当の売上を予測するモデルの作成に挑戦します。
- [スポーツのチケット価格の適正化](https://biz.quest.signate.jp/quests/10006)
    - 需給を把握した上で売上を最大化する価格設定を行うための手段として、観客数を予測するモデルの作成に挑戦します。
- [賃貸物件の家賃推定](https://biz.quest.signate.jp/quests/10060)
    - 不動産の適正価格を設定するための手段として、家賃推定モデルに挑戦します。
- [宿泊予約サービスにおける価格推定の適正化](https://biz.quest.signate.jp/quests/10011)
    - 宿泊予約サービスの売上最大化を目的として、価格の適正化に挑戦します。
- [健康経営のための疾患リスク軽減](https://biz.quest.signate.jp/quests/10012)
    - 従業員の健康管理を目的として、血液検査データをもとに個々人の疾患リスクを予測するモデルの作成に挑戦します。
- [株価を予測せよ](https://biz.quest.signate.jp/quests/10008)
    - 未来の株価を予測する課題です。時系列情報を持った回帰問題という点で当課題と共通しています。
    - Deep Learning技術を用いていることもあり、やや中上級者向けであることにご注意ください